<a href="https://colab.research.google.com/github/MrBeren/Coursera_Capstone/blob/main/Peer_graded_Assignment_Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>